### Load Packages

In [99]:
# TODO: add annotations describing usage of different modules

from operator import mod
from os import getcwd
from os.path import exists, join

import joblib
from sklearn.datasets import fetch_california_housing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler, KBinsDiscretizer
from sklearn.svm import SVR
import pandas as pd
import numpy as np
# from ydata_profiling import ProfileReport

from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.linear_model import LogisticRegression, LinearRegression
import warnings
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import  GradientBoostingClassifier
# import xgboost as xgb
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC, LinearSVC 
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold

from sklearn.metrics import recall_score

from sklearn import tree
from sklearn.decomposition import PCA, SparsePCA

from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
import json
import pickle
from IPython.display import Image
import warnings

from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from collections import Counter

import altair as alt
import random
import warnings

warnings.filterwarnings('ignore')

# set seaborn whitegrid theme
sns.set(style="whitegrid")

from sklearn.inspection import permutation_importance
from random import sample


### Load Data

In [100]:
cdc_survey = pd.read_csv('../data/cdc_nhanes_survey_responses_clean.csv')
print(cdc_survey.shape)

# filter to pregnant moms
cdc_survey_pmom = cdc_survey[cdc_survey['has_been_pregnant'] == 1]
print(cdc_survey_pmom.shape)

(35706, 863)
(7741, 863)


In [101]:
cdc_survey_pmom['weight_lbs_over_height_in_ratio'] = cdc_survey_pmom['weight_lbs'] / cdc_survey_pmom['height_in']
#cdc_survey_pmom['weight_lbs_over_height_in_ratio']

### Candidate Features

Here we list out all candidate features for our model. 

In [5]:
# Depression screener
dep_screener_cols = [
    'little_interest_in_doing_things',
    'feeling_down_depressed_hopeless',
    'trouble_falling_or_staying_asleep',
    'feeling_tired_or_having_little_energy',
    'poor_appetitie_or_overeating',
    'feeling_bad_about_yourself',
    'trouble_concentrating',
    'moving_or_speaking_to_slowly_or_fast',
    'thoughts_you_would_be_better_off_dead',
    'difficult_doing_daytoday_tasks'
]

# Alcohol & smoking
alcohol_n_smoking_cols = [
    'has_smoked_tabacco_last_5days',
    'alcoholic_drinks_past_12mo', 
    'drank_alc',
    'alc_drinking_freq',
    'alc_per_day',
    'times_with_4or5_alc',
    'times_with_8plus_alc',
    'times_with_12plus_alc',
    '4plus_alc_daily',
    'days_4plus_drinks_occasion',
    'smoked_100_cigs',
    'currently_smoke'
]

#Medical Conditions 
med_condition_cols = [
    'chest_discomfort',
    'asthma',
    'anemia_treatment',
    'blood_transfusion',
    'arthritis',
    'heart_failure',
    'coronary_heart_disease',
    'angina_pectoris',
    'heart_attack',
    'stroke',
    'thyroid_issues',
    'respiratory_issues',
    'abdominal_pain',
    'gallstones',
    'gallbladder_surgery',
    'cancer',
    'dr_recommend_lose_weight',
    'dr_recommend_exercise',
    'dr_recommend_reduce_salt',
    'dr_recommend_reduce_fat',
    'currently_losing_weight',
    'currently_increase_exercise',
    'currently_reducing_salt',
    'currently_reducing_fat',
    'metal_objects',
    'has_diabetes',    
    #Blood Pressure & Cholesterol 
    'high_bp',
    'age_hypertension',
    'hypertension_prescription',
    'high_bp_prescription',
    'high_cholesterol',
    'cholesterol_prescription',
    # general health
    'has_overweight_diagnosis',   
    'height_in',
    'weight_lbs', 
    'weight_lbs_over_height_in_ratio',
    'general_health_condition',      
]

# diet, nutrition, & exercise 
lifestyle_cols = [
    # exercise/weight loss
    'vigorous_work',
    'walk_or_bicycle',
    'vigorous_recreation',
    'moderate_recreation',    
    'count_days_moderate_recreational_activity',   
    'count_minutes_moderate_recreational_activity',
    'count_minutes_moderate_sedentary_activity',    
    'count_lost_10plus_pounds',
    'has_tried_to_lose_weight_12mo', 
    'attempt_weight_loss_1yr',           
    # diet
    'how_healthy_is_your_diet',    
    'milk_consumption_freq',
    'govmnt_meal_delivery',
    'nonhomemade_meals',
    'fastfood_meals',
    'readytoeat_meals',
    'frozen_pizza'   
]

# Reproductive Health 
reproductive_health_cols = [
    'regular_periods',
    'age_last_period',
    'try_pregnancy_1yr',
    'see_dr_fertility',
    'pelvic_infection',
    'pregnant_now',
    'pregnancy_count',
    'diabetes_pregnancy',
    'delivery_count',
    'live_birth_count',
    'age_at_first_birth',
    'age_at_last_birth',
    'months_since_birth',
    'horomones_not_bc'
]

# hospital usage & access to care
hospital_utilization_n_access_cols = [
    'general_health',
    'regular_healthcare_place',
    'time_since_last_healthcare',
    'overnight_in_hospital',
    'seen_mental_health_professional',
    'count_days_seen_doctor_12mo',
    'duration_last_healthcare_visit',   
]

# socioeconomic status
socioeconomic_cols = [
    #Food Security
    'emergency_food_received',
    'food_stamps_used',      
    #Health Insurance
    'have_health_insurance',
    'have_private_insurance',
    'plan_cover_prescriptions',   
    #Income
    'family_poverty_level',
    'family_poverty_level_category',
    #Occupation
    'hours_worked',
    'over_35_hrs_worked',
    'work_schedule'    
]

demographic_cols = [
    'food_security_level_household',   
    'food_security_level_adult',    
    'monthly_poverty_index_category',
    'monthly_poverty_index',
    'count_hours_worked_last_week',
    'age_in_years',   
    'education_level',
    'is_usa_born',    
    'has_health_insurance',
    'has_health_insurance_gap'   
]

# combine all candidate features
candidate_features = dep_screener_cols + \
    alcohol_n_smoking_cols + \
    med_condition_cols + \
    hospital_utilization_n_access_cols + \
    lifestyle_cols + \
    socioeconomic_cols + \
    reproductive_health_cols + \
    demographic_cols


len(candidate_features)

117

### Function to create test & train dataset

In [6]:
def get_model_data(original_df, 
                   columns, 
                   test_size_to_use=0.2,
                   drop_null_rows=False,
                   null_imputer_strategy='median', # mean, median, most_frequent
                   use_value_scaler=True,
                   use_smote=False):
    """
    Function to build feature & indicator matrices for both train & test.
    """
    
    # add target column (MDD)
    cols_to_use = columns.copy()
    cols_to_use.insert(0, 'MDD')
    
    df_to_use = original_df[cols_to_use]
    
    if drop_null_rows:
        df_to_use.dropna(inplace=True)
    
    # Create test & train data
    x = df_to_use.iloc[:,1:].values
    y = df_to_use['MDD'].values
    
    if not drop_null_rows:
        # SimpleImputer() = fill in missing values
        # note imputer may drop columns if no values exist for it
        imputer = SimpleImputer(strategy=null_imputer_strategy)  
        x = imputer.fit_transform(x)

    # RobustScaler() = scale features to remove outliers
    if use_value_scaler:
        trans = RobustScaler()
        x = trans.fit_transform(x)

    x_train, x_test, y_train, y_test = train_test_split(
        x, 
        y, 
        test_size=test_size_to_use, 
        random_state=42
    ) 
    
    # Technique to de-risk from positive class imbalance
    if use_smote:
        sm = SMOTE(random_state=42)
        x_train, y_train = sm.fit_resample(x_train, y_train)
    
    return x_train, x_test, y_train, y_test

### Function to get performance across different models

In [7]:
def get_performance_df(label_actual, label_pred, model_name):
    """
    Function to calculate performance metrics for model.
    Includes precision, recal, F1, & support.
    """
    # create classification report
    result_table = classification_report(label_actual, label_pred, output_dict=True)
    result_table = pd.DataFrame.from_dict(result_table)

    # store for later
    accuracies = result_table['accuracy']

    # rename grouping
    result_table.columns = [
        'depressed_no',
        'depressed_yes',
        'accuracy',
        'macro_avg',
        'weighted_avg'
    ]

    # create dataframe with 1 row per grouping
    result_table.drop(labels = 'accuracy', axis = 1, inplace=True)
    result_table = result_table.transpose()
    result_table['accuracy'] = list(accuracies)
    result_table = result_table.reset_index()
    result_table.rename(columns = {'index':'grouping'},inplace=True)
    result_table['model'] = model_name
    result_table = result_table[['model','grouping','precision','recall','f1-score','support','accuracy']]
    return result_table

def generate_models(
    x_train, 
    y_train, 
    x_test, 
    y_test):
    """
    Function that trains and makes predictions using 5 of the classifiers went over during the class.
    Meant as a helper function for easier testing of different modeling pipelines.
    """

    # K-Nearest Neighbors
    knn = KNeighborsClassifier()
    knn.fit(x_train, y_train)
    # test
    pred_labels_knn  = knn.predict(x_test)
    score_knn = get_performance_df(y_test, pred_labels_knn,'Knn')
    # train
    pred_labels_knn_train  = knn.predict(x_train)
    score_knn_train = get_performance_df(y_train, pred_labels_knn_train,'Knn')
    
    # Logistic Regression
    lm = LogisticRegression()
    lm.fit(x_train, y_train)
    # test 
    pred_labels_lr  = lm.predict(x_test)
    score_lr = get_performance_df(y_test, pred_labels_lr,'Logistic Regression')
    # train 
    pred_labels_lr_train  = lm.predict(x_train)
    score_lr_train = get_performance_df(y_train, pred_labels_lr_train,'Logistic Regression')    
        
    # Bernoulii Naive Bayes
    bnb = BernoulliNB()
    bnb.fit(x_train, y_train)
    # test 
    pred_labels_bnb  = bnb.predict(x_test)
    score_bnb = get_performance_df(y_test, pred_labels_bnb,'Bernoulli Naive Bayes')    
    # train 
    pred_labels_bnb_train  = bnb.predict(x_train)
    score_bnb_train = get_performance_df(y_train, pred_labels_bnb_train,'Bernoulli Naive Bayes')       
        
    # Gaussian Naive Bayes
    gnb = GaussianNB()
    gnb.fit(x_train, y_train)
    # test 
    pred_labels_gnb  = gnb.predict(x_test)
    score_gnb = get_performance_df(y_test, pred_labels_gnb,'Gaussian Naive Bayes')  
    # train 
    pred_labels_gnb_train  = gnb.predict(x_train)
    score_gnb_train = get_performance_df(y_train, pred_labels_gnb_train,'Gaussian Naive Bayes')         

    # Random Forest
    rf = RandomForestClassifier(random_state=0)
    rf.fit(x_train, y_train)
    # test     
    pred_labels_rf  = rf.predict(x_test)
    score_rf = get_performance_df(y_test, pred_labels_rf,'Random Forest')   
    # train 
    pred_labels_rf_train  = rf.predict(x_train)
    score_rf_train = get_performance_df(y_train, pred_labels_rf_train,'Random Forest')         
    
    #Decision Tree
    dt = DecisionTreeClassifier()
    dt.fit(x_train, y_train)
    # test
    pred_labels_dt = dt.predict(x_test)
    score_dt = get_performance_df(y_test, pred_labels_dt,'Decision Tree')
    # train 
    pred_labels_dt_train  = dt.predict(x_train)
    score_dt_train = get_performance_df(y_train, pred_labels_dt_train,'Decision Tree')          

    #Gradient Boosting Classifier
    gb = GradientBoostingClassifier()
    gb.fit(x_train, y_train)
    # test 
    pred_labels_gb = gb.predict(x_test)
    score_gb = get_performance_df(y_test, pred_labels_gb,'Gradient Boosting Classifier')
    # train 
    pred_labels_gb_train  = gb.predict(x_train)
    score_gb_train = get_performance_df(y_train, pred_labels_gb_train,'Gradient Boosting Classifier')     
    
    # make dataframe with scores (on test data)
    scores = pd.concat(
        [score_knn, 
         score_lr, 
         score_bnb, 
         score_gnb, 
         score_rf, 
         score_dt, 
         score_gb]
    )
    scores = scores.sort_values(by = 'recall', ascending=False)
    
    # make dataframe with scores (on train data)
    scores_train = pd.concat(
        [score_knn_train, 
         score_lr_train, 
         score_bnb_train, 
         score_gnb_train, 
         score_rf_train, 
         score_dt_train, 
         score_gb_train]
    )
    scores_train = scores_train.sort_values(by = 'recall', ascending=False)    
        
    # make dataframe with predictions
    predictions = pd.DataFrame({
        'actuals':y_test,
        'pred_knn':pred_labels_knn,
        'pred_logistic_regression':pred_labels_lr,
        'pred_bernoulli_naive_bayes':pred_labels_bnb,
        'pred_gaussian_naive_bayes':pred_labels_gnb,
        'pred_random_forest':pred_labels_rf,
        'pred_decision_tree':pred_labels_dt,
        'pred_gradient_boosting_classifier':pred_labels_gb
    })

    return scores, predictions, scores_train

### Function for Error analysis

In [8]:
def plot_confusion_matrix(y_test, pred_labels):
    """
    Function that displays a confusion matrix for provided true and predicted classes
    """
    #print(f'cover type 1 and type 2 total correct {np.sum(np.diag(metrics.confusion_matrix(y_test, pred_labels))[:2])}')

    cm = confusion_matrix(y_test, pred_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    fig, ax = plt.subplots(figsize=(5,5))
    disp = disp.plot(include_values=True, cmap='viridis', ax=ax, xticks_rotation='horizontal')    
    plt.grid(False)
    plt.show()
    return

def label_pred(row):
    """
    Function that labels prediction cases into TP, TN, FP, FN
    """
    if row['y_actual'] == row['y_pred']:
        if row['y_actual'] == 1:
            result = 'TP'
        else:
            result = 'TN'
    else: 
        if row['y_actual'] == 1:
            result = 'FN'
        else:
            result = 'FP'

    return result

def create_error_plots(
    y_actuals,
    y_predictions,
    x_test_matrix,
    x_test_columns,
    columns_to_plot,
    show_bar_plot=True,
    show_density_plot=True):
    """
    Function that creates bi-variate plots comparing TP, TN, FP, FN for each desired feature column.
    """
    
    ### Create dataframe of predictions & feature 
    
    # create dataframe with actuals & predictions w/ labels
    prediction_labels = pd.DataFrame(columns = ['y_actual','y_pred'])
    prediction_labels['y_actual'] = y_actuals
    prediction_labels['y_pred'] = y_predictions
    prediction_labels['label'] = prediction_labels.apply(label_pred, axis = 1)

    # feature df 
    feature_df = pd.DataFrame(x_test_matrix, columns = x_test_columns)

    # combine into 1
    pred_data = pd.concat([prediction_labels, feature_df], axis = 1)

    ### Bi-variate plots
    for column in columns_to_plot:
    
        if show_density_plot:
            ## Density plot
            sns.kdeplot(
                data=pred_data, 
                x=column, 
                hue="label", 
                cut=0, 
                common_norm=False, 
                alpha=0.4
            )

        if show_bar_plot:
            ## barplot
            # create % of total column by group
            group_data = pred_data.groupby(['label',column])['y_actual'].count().reset_index()
            group_data['Pct of Total'] = group_data['y_actual'] / group_data.groupby('label')['y_actual'].transform('sum')

            sns.catplot(
                data=group_data, 
                kind="bar",
                x=column, 
                y="Pct of Total", 
                hue="label",
                alpha = 0.8
            )

        plt.show()
        
    return pred_data

In [12]:
def get_random_forest_n_top_features(train_data=x_train, 
                                     train_labels=y_train,
                                     feature_columns=dep_screener_cols):
    """
    A model that trains a random forest model and returns the feature importances as a dataframe.
    The index of the row corresponds to the column index in the initial array
    """
    
    # fit random forest and print accuracy
    rf = RandomForestClassifier(random_state=0)
    rf.fit(train_data, train_labels)
   
    # list feature importances
    # Get numerical feature importances
    importances = list(rf.feature_importances_)
    feature_importances = pd.DataFrame({'Feature': feature_columns,'Feature Importance (RF)': importances})
    feature_importances = feature_importances.sort_values('Feature Importance (RF)',ascending=False)
    
    return feature_importances

## Baseline Model

We start by building baseline models across the 6 classifiers. We start by using the 10 questions from the depression screener questionaire.

In [13]:
x_train, x_test, y_train, y_test = get_model_data(
    original_df = cdc_survey_pmom,
    columns = dep_screener_cols
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

baseline_model, predictions, performance_train = generate_models(
    x_train, 
    y_train, 
    x_test, 
    y_test
)

baseline_model_summary = baseline_model[baseline_model['grouping'] == 'macro_avg']
baseline_model_summary[['model','recall','f1-score','precision','accuracy','support']]

(6192, 10)
(1549, 10)
(6192,)
(1549,)


,model,recall,f1-score,precision,accuracy,support
2,Bernoulli Naive Bayes,0.675232,0.632390,0.614506,0.826985,1549.0
2,Gaussian Naive Bayes,0.660506,0.633699,0.618641,0.840542,1549.0
2,Decision Tree,0.558647,0.568143,0.589451,0.867657,1549.0
2,Random Forest,0.543840,0.555191,0.688099,0.896062,1549.0
2,Logistic Regression,0.538270,0.546401,0.685605,0.896062,1549.0
2,Knn,0.534046,0.539683,0.655004,0.893480,1549.0
2,Gradient Boosting Classifier,0.519854,0.513698,0.719752,0.897999,1549.0


Naive Bayes are the best

In [306]:
# sns.catplot(
#     data=baseline_model, 
#     kind="bar",
#     x='recall', 
#     y="model", 
#     hue="grouping",
#     hue_order=['macro_avg','weighted_avg','depressed_no','depressed_yes'],
#     alpha = 0.8
# )

# pass

In [305]:
# sns.catplot(
#     data=baseline_model, 
#     kind="bar",
#     x='f1-score', 
#     y="model", 
#     hue="grouping",
#     hue_order=['macro_avg','weighted_avg','depressed_no','depressed_yes'],
#     alpha = 0.8
# )

# pass

We see low recall among the depressed_yes group (positive class). This will be the emphasis on future model enhancements.

## Error Analysis

In [304]:
# plot_confusion_matrix(
#     y_test = predictions['actuals'], 
#     pred_labels = predictions['pred_bernoulli_naive_bayes']
# )

In [303]:
# pred_data = create_error_plots(
#     y_actuals = predictions['actuals'],
#     y_predictions = predictions['pred_bernoulli_naive_bayes'],
#     x_test_matrix = x_test,
#     x_test_columns = dep_screener_cols,
#     columns_to_plot = dep_screener_cols,
#     show_density_plot=False
# )

## Feature Engineering

- 1) binning
- 2) new variable construction

##### EDA

Using https://github.com/orenscarmeli/capstone210-ppd-carmeli_chari_hand_lee/blob/main/eda/survey_data_eda_final.ipynb as reference for which variables to bin

In [187]:
### Null value total
null_value_tot = pd.DataFrame(
    cdc_survey_pmom[candidate_features].isna().sum())\
    .rename(columns={0:'num_nulls'}
).reset_index()
null_value_tot = null_value_tot.sort_values(by = 'num_nulls',ascending=False)
null_value_tot['pct_null'] = null_value_tot['num_nulls'] / cdc_survey_pmom.shape[0]
null_value_tot.rename(columns={'index':'feature'},inplace=True)

### Univariate analysis
def univariate_analysis(
    df_to_use = cdc_survey_pmom,
    columns = dep_screener_cols,
    target_col = 'MDD',
    show_plot=True):
    
    column_values = pd.DataFrame(columns = ['col_value','pct_target','num_responses','feature'])
    column_summary = pd.DataFrame(columns=['feature','corr','corr_abs','std','min','max','unique_values','target','max_minus_min'])

    # MDD coverage by feature
    for col in columns:

        # calc mean pct for target column by indicator variable
        univariate = df_to_use.groupby(col) \
               .agg({target_col:'mean', 'SEQN':'count'}) \
               .reset_index() \
               .rename(columns={col:'col_value',target_col:'pct_target','SEQN':'num_responses'})

        if show_plot:
            print(f"\n{col}\n")
            print(univariate)
        
        # aggregate into summary table
        column_traits = pd.DataFrame({
            'feature':[col],
            'corr':[df_to_use[['MDD',col]].corr().iloc[1,0]],
            'corr_abs':[abs(df_to_use[['MDD',col]].corr().iloc[1,0])],            
            'std':[univariate[univariate['num_responses'] >= 30]['pct_target'].std()],
            'min':[univariate[univariate['num_responses'] >= 30]['pct_target'].min()],
            'max':[univariate[univariate['num_responses'] >= 30]['pct_target'].max()],
            'unique_values':[univariate[univariate['num_responses'] >= 30].shape[0]],
            'target':[target_col]
        })
        column_traits['max_minus_min'] = column_traits['max'] - column_traits['min']
        column_summary = pd.concat([column_summary,column_traits])
        
        # univariate data
        univariate['feature'] = col
        column_values = pd.concat([column_values,univariate])
        
    return column_summary, column_values
        
univariate_stats, univariate_values = univariate_analysis(columns=candidate_features,show_plot=False)

univariate_summary = univariate_stats.merge(null_value_tot, left_on='feature', right_on='feature', how = 'left')

In [162]:
#univariate_summary.sort_values(by = 'corr_abs', ascending=False)

In [188]:
univariate_summary[univariate_summary['unique_values'] >= 10].sort_values(by = 'corr_abs', ascending=False)

,feature,corr,corr_abs,std,min,max,unique_values,target,max_minus_min,num_nulls,pct_null
57,weight_lbs_over_height_in_ratio,0.094843,0.094843,0.057538,0.000000,0.193548,25,MDD,0.193548,302,0.039013
56,weight_lbs,0.094245,0.094245,0.044231,0.015385,0.205882,76,MDD,0.190498,146,0.018861
112,age_in_years,0.080434,0.080434,0.039757,0.000000,0.153226,61,MDD,0.153226,0,0.000000
82,frozen_pizza,0.079178,0.079178,0.048493,0.077392,0.254237,14,MDD,0.176845,8,0.001033
72,count_minutes_moderate_sedentary_activity,0.078043,0.078043,0.031585,0.066667,0.162162,15,MDD,0.095495,2254,0.291177
94,age_last_period,-0.075339,0.075339,0.042162,0.069444,0.240000,34,MDD,0.170556,3276,0.423201
103,age_at_first_birth,-0.046757,0.046757,0.024475,0.039216,0.121951,23,MDD,0.082736,2157,0.278646
49,age_hypertension,-0.043941,0.043941,0.059984,0.032258,0.309524,40,MDD,0.277266,4598,0.593980
90,hours_worked,-0.035564,0.035564,0.035912,0.023256,0.173913,24,MDD,0.150657,4078,0.526805
111,count_hours_worked_last_week,-0.035345,0.035345,0.035912,0.023256,0.173913,24,MDD,0.150657,4079,0.526935


# Binning

In [200]:
features_to_bin = list(univariate_summary[univariate_summary['unique_values'] >= 5]['feature'])
features_to_bin

['alcoholic_drinks_past_12mo',
 'alc_drinking_freq',
 'alc_per_day',
 'times_with_4or5_alc',
 'days_4plus_drinks_occasion',
 'age_hypertension',
 'height_in',
 'weight_lbs',
 'weight_lbs_over_height_in_ratio',
 'general_health_condition',
 'general_health',
 'count_days_seen_doctor_12mo',
 'count_days_moderate_recreational_activity',
 'count_minutes_moderate_recreational_activity',
 'count_minutes_moderate_sedentary_activity',
 'how_healthy_is_your_diet',
 'nonhomemade_meals',
 'fastfood_meals',
 'readytoeat_meals',
 'frozen_pizza',
 'family_poverty_level',
 'hours_worked',
 'age_last_period',
 'pregnancy_count',
 'delivery_count',
 'live_birth_count',
 'age_at_first_birth',
 'age_at_last_birth',
 'months_since_birth',
 'monthly_poverty_index',
 'count_hours_worked_last_week',
 'age_in_years',
 'education_level']

In [210]:
# top 20 additional features; see if any of the ones that need to be binned are within them
cor_columns = candidate_features.copy()
cor_columns.append('MDD')

# find columns where top feature is 0
#df_to_use = cdc_survey_pmom[cdc_survey_pmom['feeling_down_depressed_hopeless'] == 0]
df_to_use = cdc_survey_pmom[(cdc_survey_pmom['feeling_down_depressed_hopeless'] == 0) & 
                (cdc_survey_pmom['poor_appetitie_or_overeating'] == 0) &
                (cdc_survey_pmom['trouble_falling_or_staying_asleep'] == 0)]

correlation_matrix = df_to_use[cor_columns].corr(method='pearson')
correlation_values_abs = abs(correlation_matrix['MDD']).sort_values(ascending=False)
correlation_values_abs = correlation_values_abs[correlation_values_abs.notna()]
top_additional_candidates = list(correlation_values_abs[1:20].index)
top_additional_candidates = [col for col in top_additional_candidates if col not in dep_screener_cols]

In [211]:
top_additional_candidates

['seen_mental_health_professional',
 'times_with_12plus_alc',
 'time_since_last_healthcare',
 'duration_last_healthcare_visit',
 'horomones_not_bc',
 'count_days_seen_doctor_12mo',
 'count_lost_10plus_pounds',
 'respiratory_issues',
 'arthritis',
 'age_in_years',
 'is_usa_born',
 'regular_periods',
 'high_bp',
 'metal_objects',
 'count_minutes_moderate_sedentary_activity',
 'cholesterol_prescription',
 'chest_discomfort',
 'emergency_food_received']

## Brute Force Binning

In [213]:
# create new df with bin columns from original
cdc_survey_pmom_bins = cdc_survey_pmom.copy()

###### Create bin column 

for feature in features_to_bin:

    new_column_name = feature + '_bin'

    # get non-null values per column
    feature_values = cdc_survey_pmom[feature].dropna()

    # reshape to be 1 column
    feature_values = feature_values.to_numpy().reshape([feature_values.shape[0],1])

    # create bins using estimator
    est = KBinsDiscretizer(
        n_bins=10,
        encode='ordinal', 
        strategy='uniform', 
        subsample=None
    )
    est.fit(feature_values)
    feature_values_bin = pd.DataFrame(est.transform(feature_values))

    # dataframe with binned values
    feature_values_bin.columns = [new_column_name]

    # get original
    feature_values = pd.DataFrame(feature_values)
    feature_values.columns = ['original']

    # merge bin & non-binned values together to make a lookup
    feature_values = feature_values.merge(feature_values_bin, left_index=True, right_index=True)
    feature_value_bin_lookup = feature_values.groupby(['original',new_column_name]).count().reset_index()

    # add bin column in a way that doesn't drop nulls
    cdc_survey_pmom_bins = cdc_survey_pmom_bins.merge(
        feature_value_bin_lookup, 
        left_on=feature, 
        right_on='original', 
        how = 'left'
    )

    # drop column called "original" as was only used to join
    cdc_survey_pmom_bins.drop(columns=['original'], inplace=True)  


###### create univariate bin plots and compare    
binnned_feature_names = [feature + '_bin' for feature in features_to_bin]
univariate_stats_bin, univariate_values_bin = univariate_analysis(
    df_to_use=cdc_survey_pmom_bins,
    columns=binnned_feature_names,
    show_plot=False
)

# see if it added any correlation strength

def trim_value(value):
    return value[:-4]

univariate_stats_bin['original_feature'] = univariate_stats_bin['feature'].map(trim_value)
univariate_stats_bin = univariate_stats_bin.merge(
    univariate_stats, 
    left_on='original_feature', 
    right_on='feature', 
    how = 'left')

univariate_stats_bin['corr_abs_delta'] = univariate_stats_bin['corr_abs_x'] - univariate_stats_bin['corr_abs_y']

univariate_stats_bin[['feature_x','corr_abs_x','corr_abs_y','corr_abs_delta','unique_values_x','unique_values_y']].sort_values(
    by = 'corr_abs_delta', 
    ascending=False
)

,feature_x,corr_abs_x,corr_abs_y,corr_abs_delta,unique_values_x,unique_values_y
13,count_minutes_moderate_recreational_activity_bin,0.033495,0.022947,0.010549,3,10
21,hours_worked_bin,0.046015,0.035564,0.010451,9,24
4,days_4plus_drinks_occasion_bin,0.005377,0.001035,0.004342,2,5
0,alcoholic_drinks_past_12mo_bin,0.019891,0.016936,0.002955,10,11
1,alc_drinking_freq_bin,0.019891,0.016936,0.002955,10,11
16,nonhomemade_meals_bin,0.014345,0.012005,0.002340,9,14
22,age_last_period_bin,0.077121,0.075339,0.001782,9,34
24,delivery_count_bin,0.011686,0.010345,0.001341,6,6
11,count_days_seen_doctor_12mo_bin,0.171904,0.171175,0.000730,4,6
8,weight_lbs_over_height_in_ratio_bin,0.095494,0.094843,0.000651,6,25


## Methodical Force Binning

In [214]:
# overlap?

for f in features_to_bin:
    if f in top_additional_candidates:
        print(f)

count_days_seen_doctor_12mo
count_minutes_moderate_sedentary_activity
age_in_years


In [218]:
cols = [
    'count_days_seen_doctor_12mo',
    'count_minutes_moderate_sedentary_activity',
    'age_in_years'
]

def group_by_col(df_to_use, col):

    return df_to_use.groupby(col) \
               .agg({'MDD':'mean', 'SEQN':'count'}) \
               .reset_index() \
               .rename(columns={col:'col_value','MDD':'pct_target','SEQN':'num_responses'})

**count_days_seen_doctor_12mo**

In [220]:
print(cols[0])
group_by_col(cdc_survey_pmom,cols[0])

count_days_seen_doctor_12mo


,col_value,pct_target,num_responses
0,0.0,0.142857,14
1,1.0,0.098901,91
2,2.0,0.116564,163
3,3.0,0.121019,157
4,4.0,0.156716,268
5,5.0,0.188679,53
6,6.0,0.276596,47
7,7.0,0.000000,8
8,8.0,0.100000,10
9,9.0,0.285714,7


In [227]:
cdc_survey_pmom['count_days_seen_doctor_12mo_bin'] = pd.cut(
    cdc_survey_pmom['count_days_seen_doctor_12mo'], 
    bins=[-np.inf, 1, 2, 3, 4, 5, 6, np.inf], 
    labels=[0, 1, 2 ,3, 4, 5, 6]
)

cdc_survey_pmom.groupby('count_days_seen_doctor_12mo_bin') \
    .agg({'MDD':'mean','count_days_seen_doctor_12mo':'max','SEQN':'count'}) \
    .reset_index()

,count_days_seen_doctor_12mo_bin,MDD,count_days_seen_doctor_12mo,SEQN
0,0,0.104762,1.0,105
1,1,0.116564,2.0,163
2,2,0.121019,3.0,157
3,3,0.156716,4.0,268
4,4,0.188679,5.0,53
5,5,0.276596,6.0,47
6,6,0.300000,24.0,90


**count_minutes_moderate_sedentary_activity**

In [228]:
print(cols[1])
group_by_col(cdc_survey_pmom,cols[1])

count_minutes_moderate_sedentary_activity


,col_value,pct_target,num_responses
0,0.0,0.000000,1
1,1.0,0.333333,3
2,2.0,0.000000,2
3,5.0,0.333333,3
4,8.0,0.000000,1
5,9.0,0.000000,1
6,10.0,0.083333,12
7,11.0,0.000000,1
8,15.0,0.142857,14
9,16.0,1.000000,1


In [234]:
cdc_survey_pmom['count_minutes_moderate_sedentary_activity_bin'] = pd.cut(
    cdc_survey_pmom['count_minutes_moderate_sedentary_activity'], 
    bins=[-np.inf, 270, 660, 840, np.inf], 
    labels=[0, 1, 2, 3]
)

cdc_survey_pmom.groupby('count_minutes_moderate_sedentary_activity_bin') \
    .agg({'MDD':'mean','count_minutes_moderate_sedentary_activity':'max','SEQN':'count'}) \
    .reset_index()

,count_minutes_moderate_sedentary_activity_bin,MDD,count_minutes_moderate_sedentary_activity,SEQN
0,0,0.074199,270.0,2372
1,1,0.112163,660.0,2746
2,2,0.145833,840.0,288
3,3,0.185185,1320.0,81


**age_in_years**

In [236]:
pd.set_option('display.max_rows', 1000)
#pd.reset_option('^display.', silent=True)

print(cols[2])
group_by_col(cdc_survey_pmom,cols[2])

age_in_years


,col_value,pct_target,num_responses
0,20,0.054054,37
1,21,0.020833,48
2,22,0.061538,65
3,23,0.024390,82
4,24,0.040541,74
5,25,0.000000,81
6,26,0.076923,78
7,27,0.048193,83
8,28,0.041237,97
9,29,0.010309,97


In [244]:
cdc_survey_pmom['age_in_years_bin'] = pd.cut(
    cdc_survey_pmom['age_in_years'], 
    bins=[-np.inf, 39, 49, np.inf], 
    labels=[0, 1, 2]
)

cdc_survey_pmom.groupby('age_in_years_bin') \
    .agg({'MDD':'mean','age_in_years':'max','SEQN':'count'}) \
    .reset_index()

,age_in_years_bin,MDD,age_in_years,SEQN
0,0,0.053401,39,1985
1,1,0.093137,49,1428
2,2,0.118530,80,4328


In [246]:
cdc_survey_pmom['num_dep_screener_3'] = (cdc_survey_pmom[dep_screener_cols]>=3).sum(axis=1)
cdc_survey_pmom['num_dep_screener_3_bin'] = pd.cut(
    cdc_survey_pmom['num_dep_screener_3'], 
    bins=[-np.inf, 0, 1, 3, np.inf], 
    labels=[0, 1, 2 ,3]
)

cdc_survey_pmom.groupby('num_dep_screener_3_bin') \
    .agg({'MDD':'mean','num_dep_screener_3':'max','SEQN':'count'}) \
    .reset_index()

,num_dep_screener_3_bin,MDD,num_dep_screener_3,SEQN
0,0,0.065739,0,5963
1,1,0.119420,1,896
2,2,0.249573,3,585
3,3,0.360269,10,297


In [329]:
cdc_survey_pmom['num_dep_screener_0_to_1'] = (cdc_survey_pmom[dep_screener_cols]<=1).sum(axis=1)
cdc_survey_pmom['num_dep_screener_0_to_1_bin'] = pd.cut(
     cdc_survey_pmom['num_dep_screener_0_to_1'], 
     bins=[-np.inf, 0, 2, 3, 6, 8, np.inf], 
     labels=[0, 1, 2 ,3, 4, 5]
 )

cdc_survey_pmom.groupby('num_dep_screener_0_to_1_bin') \
    .agg({'MDD':'mean','SEQN':'count'}) \
    .reset_index()

,num_dep_screener_0_to_1_bin,MDD,SEQN
0,0,0.191781,73
1,1,0.321429,84
2,2,0.413793,87
3,3,0.297030,505
4,4,0.152919,1079
5,5,0.060883,5913


In [247]:
top_additional_candidates

['seen_mental_health_professional',
 'times_with_12plus_alc',
 'time_since_last_healthcare',
 'duration_last_healthcare_visit',
 'horomones_not_bc',
 'count_days_seen_doctor_12mo',
 'count_lost_10plus_pounds',
 'respiratory_issues',
 'arthritis',
 'age_in_years',
 'is_usa_born',
 'regular_periods',
 'high_bp',
 'metal_objects',
 'count_minutes_moderate_sedentary_activity',
 'cholesterol_prescription',
 'chest_discomfort',
 'emergency_food_received']

In [310]:
cols = [
    'count_days_seen_doctor_12mo',
    'count_minutes_moderate_sedentary_activity',
    'age_in_years'
]


top_additional_candidates_bin = [
    'seen_mental_health_professional',
    'times_with_12plus_alc',
    'time_since_last_healthcare',
    'duration_last_healthcare_visit',
    'horomones_not_bc',
    'count_days_seen_doctor_12mo_bin', # new
    'count_lost_10plus_pounds',
    'respiratory_issues',
    'arthritis',
    'age_in_years_bin', # new
    'is_usa_born',
    'regular_periods',
    'high_bp',
    'metal_objects',
    'count_minutes_moderate_sedentary_activity_bin',
    'cholesterol_prescription',
    'chest_discomfort',
    'emergency_food_received',
    'num_dep_screener_3_bin' # new
]

## Does replacing to use the binned cols help?

In [311]:
x_train, x_test, y_train, y_test = get_model_data(
    original_df = cdc_survey_pmom,
    columns = dep_screener_cols + top_additional_candidates_bin
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

baseline_model, predictions, performance_train = generate_models(
    x_train, 
    y_train, 
    x_test, 
    y_test
)

baseline_model = baseline_model[baseline_model['grouping'] == 'macro_avg']
baseline_model[['model','recall','f1-score','precision','accuracy']]

(6192, 29)
(1549, 29)
(6192,)
(1549,)


,model,recall,f1-score,precision,accuracy
2,Gaussian Naive Bayes,0.711599,0.629152,0.612122,0.797289
2,Bernoulli Naive Bayes,0.671994,0.626723,0.609438,0.821175
2,Decision Tree,0.574626,0.575876,0.577219,0.846352
2,Logistic Regression,0.566119,0.586968,0.694970,0.896062
2,Knn,0.551475,0.566395,0.679741,0.894771
2,Random Forest,0.544559,0.556422,0.702868,0.897353
2,Gradient Boosting Classifier,0.544466,0.555754,0.655267,0.892189


# Learnings

- No :/ they don't improve if we substitute the non-binned sparse features with their binned counterparts

## New Variable Construction

- try creating something using the depression screeners
- try creating some generic has_issue value for medical conditions

### Combine medical conditions together

More or less same performance

In [294]:
def create_theme_col(value):
    if value in dep_screener_cols:
        return 'dep_screener'
    elif value in alcohol_n_smoking_cols:
        return 'alcohol_n_smoking'
    elif value in med_condition_cols:
        return 'med_condition'
    elif value in lifestyle_cols:
        return 'lifestyle'
    elif value in socioeconomic_cols:
        return 'socioeconomic'
    elif value in reproductive_health_cols:
        return 'reproductive_health'
    elif value in demographic_cols:
        return 'demographic'    


univariate_stats, univariate_values = univariate_analysis(columns=candidate_features,show_plot=False)
univariate_stats['feature_theme'] = univariate_stats['feature'].map(create_theme_col) 

# medical conditions that are negatively correlated (i.e as feature increases, MDD likelihood decreases)
# for all these values 1=Yes, 2=No so it means as it goes from 2->1 MDD likehood increases


med_condition_features = univariate_stats[
    (univariate_stats['feature_theme'] == "med_condition") &
    (univariate_stats['corr'] < -0.11)
].sort_values(by = 'corr_abs', ascending=False)

med_condition_features_to_combine = med_condition_features['feature']
med_condition_features

,feature,corr,corr_abs,std,min,max,unique_values,target,max_minus_min,feature_theme
0,arthritis,-0.162268,0.162268,0.070871,0.061262,0.161488,2,MDD,0.100227,med_condition
0,chest_discomfort,-0.134263,0.134263,0.066257,0.085303,0.179005,2,MDD,0.093701,med_condition
0,dr_recommend_exercise,-0.119847,0.119847,0.051314,0.065766,0.138335,2,MDD,0.072569,med_condition
0,respiratory_issues,-0.114722,0.114722,0.080711,0.090403,0.204545,2,MDD,0.114143,med_condition


In [295]:
cdc_survey_pmom['num_medical_conditions'] = (cdc_survey_pmom[med_condition_features_to_combine]==1).sum(axis=1)

univariate_stats, univariate_values = univariate_analysis(columns=['num_medical_conditions'])
univariate_stats


num_medical_conditions

   col_value  pct_target  num_responses
0          0    0.048179           3570
1          1    0.105809           2410
2          2    0.156728           1308
3          3    0.260989            364
4          4    0.280899             89


,feature,corr,corr_abs,std,min,max,unique_values,target,max_minus_min
0,num_medical_conditions,0.194159,0.194159,0.099641,0.048179,0.280899,5,MDD,0.23272


In [302]:
cdc_survey_pmom['num_non_medical_conditions'] = (cdc_survey_pmom[med_condition_features_to_combine]==2).sum(axis=1)

univariate_stats, univariate_values = univariate_analysis(columns=['num_non_medical_conditions'])
univariate_stats


num_non_medical_conditions

   col_value  pct_target  num_responses
0          0    0.208683            714
1          1    0.112603           2682
2          2    0.076768           2488
3          3    0.064793           1281
4          4    0.046875            576


,feature,corr,corr_abs,std,min,max,unique_values,target,max_minus_min
0,num_non_medical_conditions,-0.123051,0.123051,0.06433,0.046875,0.208683,5,MDD,0.161808


In [298]:
x_train, x_test, y_train, y_test = get_model_data(
    original_df = cdc_survey_pmom,
    columns = dep_screener_cols + 
              top_additional_candidates + 
              ['num_dep_screener_3_bin'] + 
              ['num_medical_conditions']
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

baseline_model, predictions, performance_train = generate_models(
    x_train, 
    y_train, 
    x_test, 
    y_test
)

baseline_model = baseline_model[baseline_model['grouping'] == 'macro_avg']
baseline_model[['model','recall','f1-score','precision','accuracy']]

(6192, 30)
(1549, 30)
(6192,)
(1549,)


,model,recall,f1-score,precision,accuracy
2,Gaussian Naive Bayes,0.715196,0.635132,0.616443,0.803744
2,Bernoulli Naive Bayes,0.683854,0.633682,0.614999,0.822466
2,Decision Tree,0.583967,0.582338,0.580833,0.843125
2,Logistic Regression,0.572049,0.594921,0.700831,0.896708
2,Gradient Boosting Classifier,0.554979,0.571715,0.691977,0.896062
2,Random Forest,0.541414,0.551437,0.694505,0.896708
2,Knn,0.540962,0.550434,0.643031,0.890897


In [321]:
x_train, x_test, y_train, y_test = get_model_data(
    original_df = cdc_survey_pmom,
    columns = dep_screener_cols + 
              top_additional_candidates + 
              ['num_dep_screener_3_bin'] + 
          #    ['num_medical_conditions'] + 
              ['num_non_medical_conditions']
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

baseline_model, predictions, performance_train = generate_models(
    x_train, 
    y_train, 
    x_test, 
    y_test
)

baseline_model = baseline_model[baseline_model['grouping'] == 'macro_avg']
baseline_model[['model','recall','f1-score','precision','accuracy']]

(6192, 30)
(1549, 30)
(6192,)
(1549,)


,model,recall,f1-score,precision,accuracy
2,Gaussian Naive Bayes,0.717714,0.639401,0.619614,0.808263
2,Bernoulli Naive Bayes,0.675139,0.628793,0.611117,0.821821
2,Logistic Regression,0.572408,0.595686,0.705578,0.897353
2,Decision Tree,0.568603,0.568412,0.568224,0.840542
2,Gradient Boosting Classifier,0.558124,0.576336,0.698369,0.896708
2,Knn,0.557405,0.574952,0.687456,0.895416
2,Random Forest,0.542134,0.552639,0.711252,0.897999


### Weight over height ratio

More or less same performance

In [18]:
cdc_survey_pmom['weight_lbs_over_height_in_ratio'] = cdc_survey_pmom['weight_lbs'] / cdc_survey_pmom['height_in']

In [331]:
x_train, x_test, y_train, y_test = get_model_data(
    original_df = cdc_survey_pmom,
    columns = dep_screener_cols + 
              top_additional_candidates + 
              ['num_dep_screener_3_bin'] + 
          #    ['num_medical_conditions'] +
              ['num_non_medical_conditions'] +
              ['weight_lbs_over_height_in_ratio']
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

baseline_model, predictions, performance_train = generate_models(
    x_train, 
    y_train, 
    x_test, 
    y_test
)

baseline_model = baseline_model[baseline_model['grouping'] == 'macro_avg']
baseline_model[['model','recall','f1-score','precision','accuracy']]

(6192, 31)
(1549, 31)
(6192,)
(1549,)


,model,recall,f1-score,precision,accuracy
2,Gaussian Naive Bayes,0.721845,0.639321,0.619785,0.805681
2,Bernoulli Naive Bayes,0.684573,0.634949,0.616103,0.823757
2,Decision Tree,0.568603,0.568412,0.568224,0.840542
2,Logistic Regression,0.565759,0.586239,0.690234,0.895416
2,Gradient Boosting Classifier,0.558843,0.577740,0.710342,0.897999
2,Knn,0.547971,0.561075,0.667504,0.893480
2,Random Forest,0.540428,0.549916,0.744126,0.899935


In [346]:
x_train, x_test, y_train, y_test = get_model_data(
 #   drop_null_rows = True,
    original_df = cdc_survey_pmom,
    columns = dep_screener_cols + 
              top_additional_candidates +
              ['num_dep_screener_3_bin'] + 
          #    ['num_medical_conditions'] +
              ['num_non_medical_conditions'] +
              ['weight_lbs_over_height_in_ratio']
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

baseline_model, predictions, performance_train = generate_models(
    x_train, 
    y_train, 
    x_test, 
    y_test
)

baseline_model = baseline_model[baseline_model['grouping'] == 'macro_avg']
baseline_model[['model','recall','f1-score','precision','accuracy']]

(6192, 31)
(1549, 31)
(6192,)
(1549,)


,model,recall,f1-score,precision,accuracy
2,Gaussian Naive Bayes,0.721845,0.639321,0.619785,0.805681
2,Bernoulli Naive Bayes,0.684573,0.634949,0.616103,0.823757
2,Decision Tree,0.573546,0.574162,0.574799,0.844416
2,Logistic Regression,0.565759,0.586239,0.690234,0.895416
2,Gradient Boosting Classifier,0.558843,0.577740,0.710342,0.897999
2,Knn,0.547971,0.561075,0.667504,0.893480
2,Random Forest,0.540428,0.549916,0.744126,0.899935


## Best Model

In [308]:
x_train, x_test, y_train, y_test = get_model_data(
    original_df = cdc_survey_pmom,
    columns = dep_screener_cols + 
              top_additional_candidates + 
              ['num_dep_screener_3_bin']
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

baseline_model, predictions, performance_train = generate_models(
    x_train, 
    y_train, 
    x_test, 
    y_test
)

baseline_model = baseline_model[baseline_model['grouping'] == 'macro_avg']
baseline_model[['model','recall','f1-score','precision','accuracy']]

(6192, 29)
(1549, 29)
(6192,)
(1549,)


,model,recall,f1-score,precision,accuracy
2,Gaussian Naive Bayes,0.719420,0.638753,0.619241,0.806327
2,Bernoulli Naive Bayes,0.679003,0.632114,0.613897,0.823757
2,Decision Tree,0.583340,0.583572,0.583807,0.846998
2,Logistic Regression,0.572408,0.595686,0.705578,0.897353
2,Gradient Boosting Classifier,0.554260,0.570370,0.681093,0.894771
2,Random Forest,0.551568,0.567581,0.734777,0.899935
2,Knn,0.543840,0.555191,0.688099,0.896062
